In [1]:

import numpy as np
from dwave.system import LeapHybridBQMSampler, LeapHybridCQMSampler
from neal import SimulatedAnnealingSampler
import pandas as pd


In [2]:
bqm_solver = LeapHybridBQMSampler(token= 'DEV-bffc84fbc5334f436b32767dfbf3ea1ea603e152') #my id, change to give like a real imput when doing the real thing

In [3]:
class Med:
    # Constructor method to initialize the car with a make, model, and year
    def __init__(self, value, weight):
        self.value = value
        self.weight = weight
    
    def give_ks(self):
        return (self.value, self.weight)
    
    

    

In [5]:

dataset = {'sleepaid': Med(100, 4),
            'retinol': Med(200, 5),
            'Aspirin': Med(250, 3),
            'Imuran': Med(1000, 5),
            "Liver albumin" : Med(924, 7),
            "Acitretin Capsules" : Med(200, 5),
            "Tretinon" : Med(230, 9)}


values = [dataset[d].value for d in dataset.keys()] 
weight = [dataset[d].weight for d in dataset.keys()] 

v= pd.DataFrame(values)
w= pd.DataFrame(weight)



In [8]:

#j in 0, 1, ..., N-1
#wj = weight, vj = value || j

# i in 0, 1, 2, ... M-1
# ci = capacity of the ies knapsack
#x_{i, j} = 1 if j is assigned to i
from dimod import BQM

def bqm_multiple_knapsack(num_med, num_trav, dataset=None, A = 2, B = 2, C= 2):
    if not dataset:
        weight = [i for i in range(num_med)]
        values = [3 * i for i in range(num_trav)]
        capacity = [4 * j + 5 for j in range(num_trav)]
    else:
        #fill this
        pass


    #for formulate the QUBO instance

    #first Constraint: Any item j can be assigned to at most one knapsack. 
    # It is possible that an item is not assigned to any knapsack.
    
    bqm = BQM('BINARY')
    
    quadratic1 = {} 

    for j in range(num_med):
        for i in range(num_trav):
            for k in range(i):
                
                quadratic1[(f"x_{i}_{j}", f"x_{k}_{j}")] = 2 * A


    #second constraint: Ensure that no knapsack’s capacity is exceeded
    linear2 = {}
    quadratic2 = {}

    offset = 0
    for i in range(num_trav):
        ci = capacity[i]
        for j, b in zip(range(num_med), range( int(np.floor( np.log2(ci) ) ))):
            wj = weight[j]

            linear2[f'x_{i}_{j}'] = (wj ** 2 - 2 * ci * wj) * B
            linear2[f'y_{i}_{b}'] = (2 ** (2 * b) - ci * 2 ** (b + 1)) * B
            quadratic2[(f'x_{i}', f'x_{j}')] = (wj * ( 2 ** (b + 1))) * B
    
        offset += (ci ** 2) * B


    #third constraint: The objective term is formulated such that our 
    # original maximization objective function is converted to a 
    # minimization problem.

    linear3 = {}
    for i in range(num_med):

        for j in range(N):
            linear3[f'x_{i}_{j}'] = (-1 * values[i]) * C
    
        bqm.add_linear(linear2, 1)
        bqm.add_linear(linear3, 1)
        bqm.add_linear(quadratic1, 1)
        bqm.add_linear(quadratic2, 1)
        bqm.add_offset(offset, 1)
    
    return bqm

    
    





In [9]:
N = 10 #number of medicines
M = 3 #number of travellers

bqm_form = bqm_multiple_knapsack(num_med= N, num_trav=M)

sampleset = bqm_solver.sample()

print(sampleset.first.sample)


TypeError: unhashable type: 'dict'